In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the CSV files
train_data_df = pd.read_csv('Released_Data/train_data.csv')
super_classes_df = pd.read_csv('Released_Data/superclass_mapping.csv')
sub_classes_df = pd.read_csv('Released_Data/subclass_mapping.csv')

# Rename
super_classes_df.rename(columns={'class': 'superclass_name'}, inplace=True)
sub_classes_df.rename(columns={'class': 'subclass_name'}, inplace=True)

# Merge the class names with the training data
train_data_df = train_data_df.merge(super_classes_df, left_on='superclass_index', right_on='index', how='left')
train_data_df = train_data_df.merge(sub_classes_df, left_on='subclass_index', right_on='index', how='left')

# Superclass_name (can replace with subclass_name depending on classification task)
train_data_df['class'] = train_data_df['subclass_name']

# Split the dataset into training and validation sets
train_df, validation_df = train_test_split(train_data_df, test_size=0.2)

# Initialize the ImageDataGenerator with EfficientNet's preprocess_input
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Image folder path
image_folder_path = 'Released_Data/train_shuffle'

def custom_generator(df, image_folder_path, batch_size):
    num_samples = len(df)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = df.iloc[offset:offset+batch_size]

            images = []
            labels = []
            for _, row in batch_samples.iterrows():
                # Load image and preprocess
                image_path = os.path.join(image_folder_path, row['image'])
                image = load_and_preprocess_image(image_path)  # Define this function as per your preprocessing needs
                images.append(image)

                # Get one-hot encoded label
                label_index = class_name_to_index[row['subclass_name']]
                label = to_categorical(label_index, num_classes=len(all_class_names))
                labels.append(label)

            # Yield the next batch of images and labels
            yield np.array(images), np.array(labels)

train_generator = custom_generator(train_df, image_folder_path=image_folder_path, batch_size=32)
validation_generator = custom_generator(validation_df, image_folder_path=image_folder_path, batch_size=32)
# Load and preprocess images for training
# train_generator = datagen.flow_from_dataframe(
#     dataframe=train_df,
#     directory=image_folder_path,
#     x_col='image',
#     y_col='class',
#     class_mode='categorical',
#     target_size=(300, 300),
#     batch_size=32
# )

# # Load and preprocess images for validation
# validation_generator = datagen.flow_from_dataframe(
#     dataframe=validation_df,
#     directory=image_folder_path,
#     x_col='image',
#     y_col='class',
#     class_mode='categorical',
#     target_size=(300, 300),
#     batch_size=32
# )

# Load pre-trained EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model
base_model.trainable = False

# Actual number of subclasses: 
all_subclass_names = sub_classes_df['subclass_name'].tolist()

# Add custom top layers
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(all_subclass_names, activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

2023-12-07 16:11:44.825866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [12]:
sub_classes_df

,index,subclass_name
0,0,"Scotch terrier, Scottish terrier, Scottie"
1,1,"African chameleon, Chamaeleo chamaeleon"
2,2,standard schnauzer
3,3,terrapin
4,4,"great grey owl, great gray owl, Strix nebulosa"
...,...,...
83,83,"robin, American robin, Turdus migratorius"
84,84,magpie
85,85,miniature schnauzer
86,86,"European gallinule, Porphyrio porphyrio"


In [9]:
# Count the number of unique subclass indices
num_subclasses = train_data_df['subclass_index'].nunique()
print("Number of unique subclasses:", num_subclasses)


Number of unique subclasses: 87


In [8]:
# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/usr/local/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/usr/local/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/usr/local/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/local/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/usr/local/anaconda3/lib/python3.11/asyncio/ba

In [ ]:
test_image_folder = 'Released_Data/test_shuffle'   # Correct path to your test images
image_files = [os.path.join(test_image_folder, img) for img in os.listdir(test_image_folder) if img.endswith('.jpg')]
test_df = pd.DataFrame(image_files, columns=['filename'])


In [ ]:
test_df.head()

,filename
0,Released_Data/test_shuffle/9733.jpg
1,Released_Data/test_shuffle/63.jpg
2,Released_Data/test_shuffle/6400.jpg
3,Released_Data/test_shuffle/823.jpg
4,Released_Data/test_shuffle/4217.jpg


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    target_size=(300, 300),
    batch_size=32,
    class_mode=None,
    shuffle=False
)


Found 12377 validated image filenames.


In [ ]:
predictions = model.predict(test_generator)


387/387 [==============================] - 1480s 4s/step


In [ ]:
np.shape(predictions)

(12377, 4)

In [ ]:
import os
# Check the directory path
print("Directory exists:", os.path.isdir(image_folder_path))

# Check the first few rows of train_df and validation_df
print("Train DataFrame:\n", train_df.head())
print("Validation DataFrame:\n", validation_df.head())

# Check the length of the DataFrames
print("Length of train_df:", len(train_df))
print("Length of validation_df:", len(validation_df))

# Check a few image file paths
sample_images = train_df['image'].head().tolist()
for img in sample_images:
    file_path = os.path.join(image_folder_path, img)
    print(f"{img} exists:", os.path.isfile(file_path))


Directory exists: True
Train DataFrame:
          image  superclass_index  subclass_index  index_x superclass_name  \
4468  4468.jpg                 2              44        2         reptile   
3498  3498.jpg                 0              28        0            bird   
6148  6148.jpg                 0              30        0            bird   
354    354.jpg                 1              31        1             dog   
4655  4655.jpg                 2              69        2         reptile   

      index_y                                      subclass_name    class  
4468       44              hognose snake, puff adder, sand viper  reptile  
3498       28                                       black grouse     bird  
6148       30                                            vulture     bird  
354        31                                           Shih-Tzu      dog  
4655       69  leatherback turtle, leatherback, leathery turt...  reptile  
Validation DataFrame:
          image  s

In [ ]:
validation_df.to_csv("/res/superclass_pred.csv", index=False)